In [1]:
# Librairie générales

import os
import sys
import importlib

path = os.getcwd()
path_src = os.path.abspath(os.path.join(path, os.pardir,"modelisation"))
sys.path.append(path_src)

import pandas as pd

In [2]:
data = pd.read_pickle("C:/Users/33623/Optimisation/Optimisation/modelisation/donnees/test/donnees.pkl")

In [3]:
data = data[['pf2','metal','larg',	'esort_1',	'esort_2']]

In [4]:
def BuilPrepro(data):

    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer
    from sklearn.linear_model import LinearRegression
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import StandardScaler
    import numpy as np


    num_feat = [f for f in data.columns[1:] if data.dtypes[f]==np.float64]
    cat_feat = [f for f in data.columns[1:] if data.dtypes[f]==object]

    if len(cat_feat)>0:
        num_prepro  = StandardScaler()
        cat_prepro  = OneHotEncoder(handle_unknown='ignore')
        prepro =  ColumnTransformer([('num',num_prepro,num_feat),('cat',cat_prepro, cat_feat)])
    else:
        num_prepro = StandardScaler()
        prepro =  ColumnTransformer([('num',num_prepro,num_feat)])

    return prepro

In [5]:
Prepro = BuilPrepro(data)

In [6]:
from lightgbm import LGBMRegressor
from sklearn.pipeline import Pipeline


pipeline = Pipeline([("preprocessor",Prepro),("model", LGBMRegressor())])


In [16]:

param = { 
    'model__colsample_bytree':0.5,
    'model__learning_rate': 0.04,
    'model__n_estimators':100
}

pipeline.set_params(**param)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['larg', 'esort_1',
                                                   'esort_2']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['metal'])])),
                ('model',
                 LGBMRegressor(colsample_bytree=0.5, learning_rate=0.04))])

In [17]:
param_list_init = [0.5,0.04,100]
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize

def tune_model(param_list):


    print(param_list)
    param_dict = { 
    'model__colsample_bytree':param_list[0],
    'model__learning_rate': param_list[1],
    'model__n_estimators':int(param_list[2])
    }

    pipeline.set_params(**param_dict)

    X = data.drop(columns=['pf2'])
    pipeline.fit(X,data['pf2'])

    y_pred = pipeline.predict(X)
    y_true = data['pf2']
    return mean_squared_error(y_true,y_pred)

def constraint_colsample_1(param_list):
    return param_list[0] - 0.49

def constraint_colsample_2(param_list):
    return 0.71- param_list[0]

def constraint_n_estimators_1(param_list):
    return int(param_list[2]) - 50

def constraint_n_estimators_2(param_list):
    return 500 - int(param_list[2])

def constraint_learning_rate_1(param_list):
    return param_list[1] - 0.02

def constraint_learning_rate_2(param_list):
    return 0.12 - param_list[1]

# Define constraints
constraints = [{'type': 'ineq', 'fun': constraint_colsample_1},
               {'type': 'ineq', 'fun': constraint_colsample_2},
               {'type': 'ineq', 'fun': constraint_n_estimators_1},
               {'type': 'ineq', 'fun': constraint_n_estimators_2},
               {'type': 'ineq', 'fun': constraint_learning_rate_1},
               {'type': 'ineq', 'fun': constraint_learning_rate_2}]


result = minimize(tune_model, x0=param_list_init, method='L-BFGS-B', constraints=constraints)


c:\Python310\lib\site-packages\scipy\optimize\_minimize.py:576: RuntimeWarning: Method L-BFGS-B cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,


[5.e-01 4.e-02 1.e+02]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000954 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 133073, number of used features: 3
[LightGBM] [Info] Start training from score 5.942981
[5.0000001e-01 4.0000000e-02 1.0000000e+02]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001073 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 133073, number of used features: 3
[LightGBM] [Info] Start training from score 5.942981
[5.000000e-01 4.000001e-02 1.000000e+02]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Inf

In [18]:
print("Optimal Parameters:", result.x)
print("Best Accuracy:", -result.fun)

Optimal Parameters: [  0.5          1.03997454 100.        ]
Best Accuracy: -0.21364885374642018


In [12]:
tune_model(param_list_init)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000979 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 133073, number of used features: 3
[LightGBM] [Info] Start training from score 5.942981


0.35085073332083766

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('num', StandardScaler(),
                                    ['larg', 'esort_1', 'esort_2']),
                                   ('cat', OneHotEncoder(handle_unknown='ignore'),
                                    ['metal'])])),
  ('model', LGBMRegressor(colsample_bytree=0.5, learning_rate=0.04))],
 'verbose': False,
 'preprocessor': ColumnTransformer(transformers=[('num', StandardScaler(),
                                  ['larg', 'esort_1', 'esort_2']),
                                 ('cat', OneHotEncoder(handle_unknown='ignore'),
                                  ['metal'])]),
 'model': LGBMRegressor(colsample_bytree=0.5, learning_rate=0.04),
 'preprocessor__n_jobs': None,
 'preprocessor__remainder': 'drop',
 'preprocessor__sparse_threshold': 0.3,
 'preprocessor__transformer_weights': None,
 'preprocessor__transformers': [('num',
   StandardScaler(),
   ['larg', 'esort_1', 'esort_2']),
  ('cat', OneHo